In [3]:
# https://nbconvert.readthedocs.io/en/latest/removing_cells.html
# use these magic spells to update your classes methods on-the-fly as you edit them:
%reload_ext autoreload
%autoreload 2
from pprint import pprint
from IPython.core.display import display, HTML, Markdown
import ipywidgets as widgets
# %run includeme.ipynb # include a notebook from this same directory
display(HTML("<style>.container { width:100% !important; }</style>"))

/tmp/ipykernel_19959/1348678174.py:6: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML, Markdown


In [4]:
# common libs
import math, os, io, json, cv2, random, logging
import numpy as np
# images
from PIL import Image
import matplotlib.pyplot as plt

In [5]:
# fiftyone
import fiftyone as fo
import fiftyone.zoo as foz

In [6]:
# CompressAI-Vision
from compressai_vision.conversion import imageIdFileList

In [7]:
homie=os.path.expanduser("~")
print("your home path is", homie)
fodir=os.path.join(homie,'fiftyone')
print("fiftyone dowloads data to", fodir)
try:
    os.mkdir(fodir)
except FileExistsError:
    pass

your home path is /home/sampsa
fiftyone dowloads data to /home/sampsa/fiftyone


List all datasets (already) registered to fiftyone

In [ ]:
fo.list_datasets()

Create a list of necessary images

MPEG VCM working group and Nokia provides you with files listing the necessary images for detection and segmentation validation, namely ``detection_validation_input_5k.lst`` and ``segmentation_validation_input_5k.lst``.  Let's combine those two files into a list:

In [8]:
# TODO: edit according to your paths
path="/home/sampsa/silo/interdigital/siloai-playground/sampsa/nokia/"
det_lst=os.path.join(path, "data5K/detection_validation_input_5k.lst") # the images used for detection validation
seg_lst=os.path.join(path, "data5K_seg/segmentation_validation_input_5k.lst") # the images used for segmentation validation
assert(os.path.exists(det_lst)), "missing file "+det_lst
assert(os.path.exists(seg_lst)), "missing file "+seg_lst
lis=imageIdFileList(det_lst, seg_lst)
print(len(lis))

8189


Tell fiftyone to load the correct subset of OpenImageV6 dataset:

In [9]:
# https://voxel51.com/docs/fiftyone/user_guide/dataset_zoo/datasets.html#dataset-zoo-open-images-v6
dataset = foz.load_zoo_dataset(
    "open-images-v6",
    split="validation",
    # label_types=("detections", "classifications", "relationships", "segmentations") # this is the default
    image_ids=lis
)

Necessary images already downloaded
Existing download of split 'validation' is sufficient
Loading existing dataset 'open-images-v6-validation'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


In [10]:
# take a look at the dataset
dataset

Name:        open-images-v6-validation
Media type:  image
Num samples: 8189
Persistent:  True
Tags:        []
Sample fields:
    id:              fiftyone.core.fields.ObjectIdField
    filepath:        fiftyone.core.fields.StringField
    tags:            fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:        fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    positive_labels: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classifications)
    negative_labels: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classifications)
    detections:      fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    open_images_id:  fiftyone.core.fields.StringField
    relationships:   fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    segmentations:   fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)

In [11]:
# make dataset persistent .. next time you import fiftyone it's still available (loaded into the mongodb that's running in the background)
dataset.persistent=True

In [12]:
# next time you need it, load it with:
dataset = fo.load_dataset("open-images-v6-validation")

In [13]:
# peek at first sample
dataset.first()

<Sample: {
    'id': '62f17b8486a5a296ef346d7b',
    'media_type': 'image',
    'filepath': '/home/sampsa/fiftyone/open-images-v6/validation/data/0001eeaf4aed83f9.jpg',
    'tags': BaseList(['validation']),
    'metadata': None,
    'positive_labels': <Classifications: {
        'classifications': BaseList([
            <Classification: {
                'id': '62f17b8486a5a296ef346d79',
                'tags': BaseList([]),
                'label': 'Airplane',
                'confidence': 1.0,
                'logits': None,
            }>,
        ]),
        'logits': None,
    }>,
    'negative_labels': <Classifications: {'classifications': BaseList([]), 'logits': None}>,
    'detections': <Detections: {
        'detections': BaseList([
            <Detection: {
                'id': '62f17b8486a5a296ef346d7a',
                'attributes': BaseDict({}),
                'tags': BaseList([]),
                'label': 'Airplane',
                'bounding_box': BaseList([
          

Let's take a look where fiftyone downloaded the files

In [14]:
dir_=os.path.join(fodir,"open-images-v6")
print("contents of", dir_,":")
!tree --filelimit=10 $dir_ | cat

contents of /home/sampsa/fiftyone/open-images-v6 :
/home/sampsa/fiftyone/open-images-v6
├── info.json
└── validation
    ├── data [8189 entries exceeds filelimit, not opening dir]
    ├── labels
    │   ├── classifications.csv
    │   ├── detections.csv
    │   ├── masks [16 entries exceeds filelimit, not opening dir]
    │   ├── relationships.csv
    │   └── segmentations.csv
    └── metadata
        ├── attributes.csv
        ├── classes.csv
        ├── hierarchy.json
        ├── image_ids.csv
        └── segmentation_classes.csv

5 directories, 10 files


In [15]:
## if you'd like to remove it, do this:
## CAREFULL 
# fo.delete_dataset("open-images-v6-validation")

visualize the dataset

In [16]:
## starting the visualization "app" is as easy as this:
# session = fo.launch_app(dataset)